In [8]:
import numpy as np
import time
import gc
import dill as pkl
import scipy
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.ndimage import convolve
from IPython.display import HTML

In [4]:
def initialize_grid(size):
    """
    Initializes a random grid of random size
    """
    grid = np.zeros(size)
    grid = np.random.rand(*size)
    grid = np.array(grid >= np.random.rand())

    return grid.astype(np.uint8)

def expand_grid_if_needed(grid):
    """
    Makes it possible for the cells to grow beyond the initially specified grid boundaries
    """
    if np.any(grid[0, :]) or np.any(grid[-1, :]) or np.any(grid[:, 0]) or np.any(grid[:, -1]):
        # Expand grid by adding a border of dead cells
        new_grid = np.zeros((grid.shape[0] + 2, grid.shape[1] + 2), dtype=int)
        new_grid[1:-1, 1:-1] = grid
        return new_grid
    return grid

def update_grid(grid):
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    
    # Count neighbors by convolving the grid with the kernel
    neighbor_count = convolve(grid, kernel, mode="constant", cval=0)

    # Apply the Game of Life rules
    new_grid = (neighbor_count == 3) | ((grid == 1) & (neighbor_count == 2))
    
    return new_grid.astype(np.uint8)
    

def pad_and_sparse(data):
    """
    pad states to equal sizes, since grid size can grow (see function expand_grid_if_needed above)
    """
    data_padded = []
    for state in data:
        pad_x = (data[-1].shape[0] - state.shape[0]) // 2
        pad_y = (data[-1].shape[1] - state.shape[1]) // 2
        padded = np.pad(state, ((pad_x, pad_x), (pad_y, pad_y)), mode='constant', constant_values='0')
        data_padded.append(scipy.sparse.csr_matrix(padded))

    return data_padded


def run_simulation(n=0, save=False, grid=None, size=(30, 30), steps=500):
    if grid is None:
        grid = initialize_grid(size)

    data = [grid]
    
    for _ in range(steps):
        grid = expand_grid_if_needed(grid)
        grid = update_grid(grid)
        data.append(grid)
    
    if save:
        with open(f'data/sim_{n}.pkl', 'wb') as f:
            pkl.dump(pad_and_sparse(data), f)
            
    else:
        return pad_and_sparse(data)

In [6]:
# Create data

N = 10000
STEPS = 300
INIT_SIZE = (30, 30)
sims = []

for n in tqdm(range(N)):
    run_simulation(n=n, save=True)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [3:43:53<00:00,  1.34s/it]


In [ ]:
# load simluations from data folder
# requires ca 10 GB of RAM, may lead to memory overflow!

all_sims = []

for sim_path in tqdm(glob.glob('data/*.pkl')):
    with open(sim_path, 'rb') as f:
        sim = pkl.load(f)
        sim = np.array([S.A for S in sim])
    all_sims.append(sim)

In [ ]:
# Animate

some_sim = all_sims[10]

fig, ax = plt.subplots()
ax.set_title("Conway's Game of Life")
img = ax.imshow(some_sim[0], cmap='binary')

def update(frame):
    img.set_array(some_sim[frame])
    return img,

ani = animation.FuncAnimation(fig, update, frames=len(some_sim), interval=100)
HTML(ani.to_jshtml())